# Clustering - Proiect de consultanță - Soluție 

O firmă de tehnologie a fost atacată de hacker-i. Inginerii firmei au reușit să strângă date valoroase despre atacuri, incluzând durata sesiunii, locații, viteza de tastare (wpm - words per minute) etc. Inginerul relatează ce a obținut până acum: metadate ale fiecărei sesiuni prin care hacker-ii s-au conectat la servere. Caracteristicile datelor sunt următoarele:

* 'Session_Connection_Time': durata sesiunii (în minute)
* 'Bytes Transferred': Numărul de MB transferați în timpul sesiunii
* 'Kali_Trace_Used': Indică dacă hacker-ul a utilizat Kali Linux
* 'Servers_Corrupted': Numărul de servere corupte în timpul atacului
* 'Pages_Corrupted': Numărul de pagini accesate ilegal
* 'Location': Locația de la care a provenit atacul (Probabil nu este relevantă, deoarece hacker-ii au folosit VPN-uri)
* 'WPM_Typing_Speed': Viteza de tastare estimată, pe baza log-urilor sesiunilor.


Firma de tehnologie suspectează 3 potențiali hacker-i care au comis atacul. Sunt siguri de 2 dintre ei, dar nu și de al treilea.
Dorim să aflăm dacă cel de-al treilea suspect a fost implicat în aceste atacuri, sau dacă au fost implicați doar cei 2. Probabil nu se va putea spune cu siguranță, dar putem încerca să aplicăm tehnica de clustering ca să aflăm.

**O ultimă observație: inginerul știe că hacker-ii schimbă atacurile între ei, aceasta însemnând că ar trebui să aibă cam același număr de atacuri. De exemplu, dacă au fost 100 de atacuri în total, atunci în cazul cu 2 hackeri fiecare ar trebui să fi avut aproximativ 50, iar în cazul cu 3 hackeri  fiecare ar fi avut aproximativ 33. Inginerul crede că acesta este un element cheie pentru a rezolva problema, dar nu știe cum poate distinge datele neetichetate în grupuri de hackeri.**

In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('hack_find').getOrCreate()

In [21]:
from pyspark.ml.clustering import KMeans

# Încărcare date.
dataset = spark.read.csv("hack_data.csv",header=True,inferSchema=True)

In [3]:
dataset.head()

Row(Session_Connection_Time=8.0, Bytes Transferred=391.09, Kali_Trace_Used=1, Servers_Corrupted=2.96, Pages_Corrupted=7.0, Location='Slovenia', WPM_Typing_Speed=72.37)

In [4]:
dataset.describe().show()

+-------+-----------------------+------------------+------------------+-----------------+------------------+-----------+------------------+
|summary|Session_Connection_Time| Bytes Transferred|   Kali_Trace_Used|Servers_Corrupted|   Pages_Corrupted|   Location|  WPM_Typing_Speed|
+-------+-----------------------+------------------+------------------+-----------------+------------------+-----------+------------------+
|  count|                    334|               334|               334|              334|               334|        334|               334|
|   mean|     30.008982035928145| 607.2452694610777|0.5119760479041916|5.258502994011977|10.838323353293413|       null|57.342395209580864|
| stddev|     14.088200614636158|286.33593163576757|0.5006065264451406| 2.30190693339697|  3.06352633036022|       null| 13.41106336843464|
|    min|                    1.0|              10.0|                 0|              1.0|               6.0|Afghanistan|              40.0|
|    max|           

In [6]:
dataset.columns

['Session_Connection_Time',
 'Bytes Transferred',
 'Kali_Trace_Used',
 'Servers_Corrupted',
 'Pages_Corrupted',
 'Location',
 'WPM_Typing_Speed']

In [22]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [8]:
feat_cols = ['Session_Connection_Time', 'Bytes Transferred', 'Kali_Trace_Used',
             'Servers_Corrupted', 'Pages_Corrupted','WPM_Typing_Speed']

In [9]:
vec_assembler = VectorAssembler(inputCols = feat_cols, outputCol='features')

In [10]:
final_data = vec_assembler.transform(dataset)

In [11]:
from pyspark.ml.feature import StandardScaler

In [12]:
scaler = StandardScaler(inputCol="features", outputCol="scaledFeatures", withStd=True, withMean=False)

In [13]:
# Calculul statisticilor de sumarizare prin aplicarea StandardScaler
scalerModel = scaler.fit(final_data)

In [14]:
# Normalizarea fiecărei caracteristici astfel încât să aibă deviația standard unitară.
cluster_final_data = scalerModel.transform(final_data)

** Încercăm să aflăm dacă au fost 2 sau 3 hackeri! **

In [15]:
kmeans3 = KMeans(featuresCol='scaledFeatures',k=3)
kmeans2 = KMeans(featuresCol='scaledFeatures',k=2)

In [16]:
model_k3 = kmeans3.fit(cluster_final_data)
model_k2 = kmeans2.fit(cluster_final_data)

In [17]:
#Depreciat
#wssse_k3 = model_k3.computeCost(cluster_final_data)
#wssse_k2 = model_k2.computeCost(cluster_final_data)

#Evaluați modelele!

In [197]:
#Depreciat
#print("With K=3")
#print("Within Set Sum of Squared Errors = " + str(wssse_k3))
#print('--'*30)
#print("With K=2")
#print("Within Set Sum of Squared Errors = " + str(wssse_k2))

#Afisati evaluarea modelelor

With K=3
Within Set Sum of Squared Errors = 434.1492898715845
------------------------------------------------------------
With K=2
Within Set Sum of Squared Errors = 601.7707512676716


Putem continua analiza observând diferența între K=3 și K=4.

In [18]:
for k in range(2,9):
    kmeans = KMeans(featuresCol='scaledFeatures',k=k)
    model = kmeans.fit(cluster_final_data)
    
    #Evaluati modelul
    #wssse = model.computeCost(cluster_final_data)
    
    print("With K={}".format(k))
    #Afisati evaluarea
    #print("Within Set Sum of Squared Errors = " + str(wssse))
    print('--'*30)

With K=2
------------------------------------------------------------
With K=3
------------------------------------------------------------
With K=4
------------------------------------------------------------
With K=5
------------------------------------------------------------
With K=6
------------------------------------------------------------
With K=7
------------------------------------------------------------
With K=8
------------------------------------------------------------


** Probabil analiza de mai sus nu arată nimic relevant. Folosim, însă, ultima observație din enunț, legată de egalitatea numărului de atacuri între hacker-i. Verificăm acest lucru, aplicând modelele și grupând după predicție**

In [19]:
model_k3.transform(cluster_final_data).groupBy('prediction').count().show()

+----------+-----+
|prediction|count|
+----------+-----+
|         1|   79|
|         2|   88|
|         0|  167|
+----------+-----+



In [20]:
model_k2.transform(cluster_final_data).groupBy('prediction').count().show()

+----------+-----+
|prediction|count|
+----------+-----+
|         1|  167|
|         0|  167|
+----------+-----+



________

### Concluzia: au fost 2 hackeri. Algoritmul de clustering a creat 2 clustere de dimensiune egală în cazul K=2.

